# 如何在构造函数中传递回调

:::info 前置条件

本指南假设您已熟悉以下概念：

- [回调](/docs/concepts/callbacks)
- [自定义回调处理程序](/docs/how_to/custom_callbacks)

:::

大多数 LangChain 模块允许您直接将 `callbacks` 传递到构造函数（即初始化器）中。在这种情况下，回调将仅针对该实例（及其任何嵌套运行）调用。

:::warning
构造函数回调仅限于定义它们的对象范围内。它们**不会**被对象的子对象继承。这可能会导致令人困惑的行为，
因此通常更好地将回调作为运行时参数传递。
:::

以下是一个示例：

In [1]:
# | output: false
# | echo: false

%pip install -qU langchain langchain_anthropic

import getpass
import os

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

In [ ]:
from typing import Any, Dict, List

from langchain_anthropic import ChatAnthropic
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate


class LoggingHandler(BaseCallbackHandler):
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("聊天模型已启动")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"聊天模型已结束，响应：{response}")

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        print(f"链 {serialized.get('name')} 已启动")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"链已结束，输出：{outputs}")


callbacks = [LoggingHandler()]
llm = ChatAnthropic(model="claude-3-sonnet-20240229", callbacks=callbacks)
prompt = ChatPromptTemplate.from_template("1 + {number} 等于多少？")

chain = prompt | llm

chain.invoke({"number": "2"})

Chat model started
Chat model ended, response: generations=[[ChatGeneration(text='1 + 2 = 3', message=AIMessage(content='1 + 2 = 3', response_metadata={'id': 'msg_01CdKsRmeS9WRb8BWnHDEHm7', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 16, 'output_tokens': 13}}, id='run-2d7fdf2a-7405-4e17-97c0-67e6b2a65305-0'))]] llm_output={'id': 'msg_01CdKsRmeS9WRb8BWnHDEHm7', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 16, 'output_tokens': 13}} run=None


AIMessage(content='1 + 2 = 3', response_metadata={'id': 'msg_01CdKsRmeS9WRb8BWnHDEHm7', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 16, 'output_tokens': 13}}, id='run-2d7fdf2a-7405-4e17-97c0-67e6b2a65305-0')

您可以看到我们仅看到来自聊天模型运行的事件——没有来自提示或更广泛链的链事件。

## 下一步

您现在已经了解了如何将回调传递到构造函数中。

接下来，请查看本节中的其他操作指南，例如如何[在运行时传递回调](/docs/how_to/callbacks_runtime)。